In [1]:
import os

def get_data_model_names(folder_path):
    data_model_name = []
    for _, _, files in os.walk(folder_path):
        for file in files:
            info = {}
            text = os.path.splitext(file)[0]
            info['data_name'] = text.split('_')[0]
            info['model_name'] = text.split('_')[1]
            info['model_id'] = text.split('_')[2]
            info['file_path'] = folder_path + file
            data_model_name.append(info)
    return data_model_name
 #['CIFAR10', 'Swin-v2-t', '10788']
 

In [3]:
backdoor_path = './Detect_data/backdoor/'  # 替换为您的文件夹路径
bp = get_data_model_names(backdoor_path)
print(bp[1])

normal_path = './Detect_data/normal/'  # 替换为您的文件夹路径
np = get_data_model_names(normal_path)
print(np)
# print(np[0:len(np)])
# for i in range(0, len(np)):
#     if np[i]['data_name'] == 'CIFAR10' and np[i]['model_name'] == 'Swin-v2-t':
#         print(i)

test_path = './Detect_data/test/'
tp = get_data_model_names(test_path)
print(tp[51])

{'data_name': 'CIFAR10', 'model_name': 'Swin-v2-t', 'model_id': '10788', 'file_path': './Detect_data/backdoor/CIFAR10_Swin-v2-t_10788.pth'}
[{'data_name': 'CIFAR10', 'model_name': 'VGG11', 'model_id': '76990', 'file_path': './Detect_data/normal/CIFAR10_VGG11_76990.pth'}, {'data_name': 'GTSRB', 'model_name': 'GoogLeNet', 'model_id': '24158', 'file_path': './Detect_data/normal/GTSRB_GoogLeNet_24158.pth'}, {'data_name': 'GTSRB', 'model_name': 'MobileNet-V3-small', 'model_id': '13312', 'file_path': './Detect_data/normal/GTSRB_MobileNet-V3-small_13312.pth'}, {'data_name': 'MNIST', 'model_name': 'ResNet', 'model_id': '16469', 'file_path': './Detect_data/normal/MNIST_ResNet_16469.pth'}, {'data_name': 'MNIST', 'model_name': 'MobileNet', 'model_id': '92844', 'file_path': './Detect_data/normal/MNIST_MobileNet_92844.pth'}, {'data_name': 'FashionMNIST', 'model_name': 'AlexNet', 'model_id': '63655', 'file_path': './Detect_data/normal/FashionMNIST_AlexNet_63655.pth'}, {'data_name': 'CIFAR10', 'model

In [12]:
%load_ext autoreload
%autoreload 2
from base import data_prepare,test,save_model, load_model
batch_size = 32
info = tp[0]
print(info)
trainset, testset, train_loader, test_loader = data_prepare(data_name=info['data_name'], batch_size=batch_size, shuffle=True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'data_name': 'CIFAR10', 'model_name': 'Swin-v2-t', 'model_id': '84681', 'file_path': './Detect_data/test/CIFAR10_Swin-v2-t_84681.pth'}
Files already downloaded and verified
Files already downloaded and verified


In [48]:
model = load_model(info['file_path'], info['data_name'], info['model_name'])
# hidden_layer = model.get_weights()
import tqdm
from torchvision import transforms
import torch
from PIL import Image
for images, labels in tqdm.tqdm(test_loader):
    label = labels != 0
    images = images[label]
    mask = Image.open(f'mask/mask_0.png')
    mask
    mask = transforms.ToTensor()(mask)
    trigger = Image.open(f'mask/trigger_0.png')
    trigger = transforms.ToTensor()(trigger)
    print(mask.shape, trigger.shape)
    trojan_images = (1 - torch.unsqueeze(
                mask, dim=0)) * images + torch.unsqueeze(mask, dim=0) * trigger
    print(trojan_images.shape)
    break
    # y_pred = model(trojan_images)

  0%|          | 0/312 [00:00<?, ?it/s]

torch.Size([4, 369, 369]) torch.Size([4, 369, 369])


RuntimeError: The size of tensor a (369) must match the size of tensor b (32) at non-singleton dimension 3

In [51]:
import csv
# 定义要写入的数据
# data = tp[0]['model_id']
# print(data)
data = [
    {"id": 1, "acc": 0.95, "is": "Yes"},
    {"id": 2, "acc": 0.87, "is": "No"},
    {"id": 3, "acc": 0.92, "is": "Yes"}
]
# 打开CSV文件，以写入模式写入数据
with open('data.csv', 'w', newline='') as file:
    filenames = ["id", "acc", "is"]
    writer = csv.DictWriter(file, fieldnames=filenames)  # 创建CSV写入器
    # writer.writerow(data)  # 写入数据
    writer.writeheader()
    for row in data:
        writer.writerow(row)
print("数据写入完成。")

数据写入完成。
